# Spectrogram data preparation

Aim: create a pipeline which creates a dataset of (spectrogram, caption) pairs

1. Load from huggingface
2. Inspect data
3. Prepare dataset of first speaker


In [ ]:
import torchaudio
import librosa
import librosa.display
import numpy as np
from torchaudio.transforms import Spectrogram
from torchvision.transforms import functional as tf
import matplotlib.pyplot as plt

In [ ]:
# Check available datasetes

from datasets import load_dataset, list_datasets

dataset_lists = list_datasets()

print(", ".join(dataset for dataset in dataset_lists))

In [ ]:
dataset = load_dataset("Loie/VGGSound", split="train")

In [ ]:
print(dataset)
dataset[0]

Mel spectrogram conversion

In [ ]:
import torchaudio
import torchaudio.transforms as transforms


# Takes an existing dataset and creates a copy with a spectrogram columm - no prefix
def transform_wav(dataset):
    transformed_data = []

    for data in dataset:
        waveform, sample_rate = torchaudio.load(data["audio"]["path"], normalize=True)

        # Parameters: number of fourier bins & mel filter banks
        n_fft = 256
        n_mels = 80

        transform = transforms.MelSpectrogram(
            sample_rate=sample_rate, n_fft=n_fft, n_mels=n_mels
        )

        spectrogram = transform(waveform)

        data["spectrogram"] = spectrogram
        transformed_data.append(data)

    return transformed_data


# Takes an existing dataset and creates a copy with a spectrogram column, allows selection of prefix
def transform_wav(dataset, prefix):
    prefix_len = len(prefix)
    filtered_data = [d for d in dataset if d["id"][:prefix_len] == prefix]

    transformed_data = []

    for data in filtered_data:
        waveform, sample_rate = torchaudio.load(data["audio"]["path"], normalize=True)

        # Parameters: number of fourier bins & mel filter banks
        n_fft = 256
        n_mels = 80

        transform = transforms.MelSpectrogram(
            sample_rate=sample_rate, n_fft=n_fft, n_mels=n_mels
        )

        spectrogram = transform(waveform)

        data["spectrogram"] = spectrogram
        transformed_data.append(data)

    return transformed_data


""" 
Transformation back into
invers_transform = torchaudio.transforms.InverseMelScale(sample_rate=sample_rate, n_stft=n_stft)
grifflim_transform = torchaudio.transforms.GriffinLim(n_fft=n_fft)

mel_specgram = transform(waveform)
inverse_waveform = invers_transform(mel_specgram)
pseudo_waveform = grifflim_transform(inverse_waveform) """

test_dataset = transform_wav(dataset, "LJ001")

In [ ]:
# Convert tf records to pytorch tensors